<a href="https://colab.research.google.com/github/JRamos84/deteccion_fraude/blob/main/notebooks/03_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 03 Feature Selection
**Objetivo**: Seleccionar las características más relevantes para el modelo.
**Contenido**:
- Técnicas de selección de características (e.g., selección univariante, selección recursiva, modelos basados en árboles).
- Evaluación de la importancia de las características.

In [23]:
## Importar librerrias
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder


In [24]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# Guardar X y y en archivos CSV

#dir_data_processed = '../data/processed'
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_X_train = os.path.join(dir_data_processed, 'X__train_procesados.csv')
ruta_archivo_X_test = os.path.join(dir_data_processed, 'X_test_procesados.csv')
ruta_archivo_X_val = os.path.join(dir_data_processed, 'X_val_procesados.csv')

ruta_archivo_y_train = os.path.join(dir_data_processed, 'y_train_procesados.csv')
ruta_archivo_y_test = os.path.join(dir_data_processed, 'y_test_procesados.csv')
ruta_archivo_y_val = os.path.join(dir_data_processed, 'y_val_procesados.csv')


X_train_final= pd.read_csv(ruta_archivo_X_train)
X_val_final= pd.read_csv(ruta_archivo_X_val)
X_test_final= pd.read_csv(ruta_archivo_X_test)

y_train_final= pd.read_csv(ruta_archivo_y_train)
y_val_final= pd.read_csv(ruta_archivo_y_val)
y_test_final= pd.read_csv(ruta_archivo_y_test)



### Recuperar nombres de columnas de categoría y numéricas

In [26]:
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_columns = os.path.join(dir_data_processed, 'columns.pkl')
with open(ruta_archivo_columns, 'rb') as file:
    num_cols, cat_cols = pickle.load(file)

print('num_cols:', num_cols)
print('cat_cols:', cat_cols)


num_cols: ['TransactionID', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D10', 'D15', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137

## chi-cuadrado

In [27]:

chi2_selector = SelectKBest(chi2, k=20)
X_kbest = chi2_selector.fit_transform(X_train_final, y_train_final)
col_selected_catg = X_train_final.columns[chi2_selector.get_support()].tolist()
print("Características seleccionadas:", col_selected_catg)


Características seleccionadas: ['V94', 'V103', 'V279', 'V280', 'V281', 'V292', 'V293', 'V294', 'V295', 'V312', 'V94.1', 'V103.1', 'V279.1', 'V280.1', 'V281.1', 'V292.1', 'V293.1', 'V294.1', 'V295.1', 'V312.1']


usando el metodo chi2 para seleccionar las características más relevante son:
 - 'V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320'

-  'card4_discover', 'card6_credit', 'card6_debit', 'M2_F', 'M3_F', 'M3_T',M6_F', 'M6_T', 'P_emaildomain_0', 'P_emaildomain_2'

## Anova

In [28]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selección de las 10 mejores características usando ANOVA
anova_selector = SelectKBest(f_classif, k=20)
X_kbest = anova_selector.fit_transform(X_train_final[num_cols], y_train_final)
col_selected_num = X_train_final[num_cols].columns[anova_selector.get_support()].tolist()
print("Características seleccionadas:", col_selected_num)



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Características seleccionadas: ['V95', 'V97', 'V101', 'V103', 'V128', 'V279', 'V280', 'V281', 'V282', 'V283', 'V289', 'V292', 'V293', 'V295', 'V307', 'V308', 'V312', 'V315', 'V317', 'V318']


In [29]:

# Guardar las listas en un archivo pickle
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_columns = os.path.join(dir_data_processed, 'columns_feature_selection.pkl')
with open(ruta_archivo_columns, 'wb') as file:
    pickle.dump((col_selected_catgl, col_selected_catg), file)

NameError: name 'col_selected_catgl' is not defined

## Conclusion

- se uso el metodo chi2 para seleccionar las categorías más relevantes:
 - 'V135', 'V319', 'V137', 'V321', 'V129', 'V132', 'V309', 'V136', 'V316',
'V320'
 - 'card4_discover', 'card6_credit', 'card6_debit', 'M2_F', 'M3_F', 'M3_T',M6_F', 'M6_T', 'P_emaildomain_0', 'P_emaildomain_2'
- El metodo anova para las categorias numericas.
  'V313', 'V134', 'V318', 'V315', 'V306', 'D2', 'V317', 'V128', 'V308',
'V307'